In [128]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QTableWidget, QTableWidgetItem, QVBoxLayout, QWidget
import tkinter as tk
from tkinter import *
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import load_model
import joblib
import psycopg2
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Dense
import os
from torch.utils.data import Dataset, DataLoader,TensorDataset
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from pickle import dump


In [129]:
def submit_values(window,values1,values2, values3, values4, values5, values6, values7, values8, values9, values10, values11, values12, values13, values14):
    values1 = int(values1.get())
    values2 = int(values2.get())
    values3 = int(values3.get())
    values4 = int(values4.get())
    values5 = int(values5.get())
    values6 = int(values6.get())
    values7 = int(values7.get())
    values8 = int(values8.get())
    values9 = int(values9.get())
    values10 = int(values10.get())
    values11 = float(values11.get())
    values12 = int(values12.get())
    values13 = int(values13.get())
    values14 = int(values14.get())

    Values = [values2, values3, values4, values5, values6, values7, values8, values9, values10, values11, values12, values13, values14]
    data1 =[values1,values2,values3]
    data2 = [values4,values5,values6,values7,values8,values9,values10,values11,values12,values13,values14,values1]
    evaluation(Values,data1,data2)
    
   

In [130]:
def createUI():
    global main_window, values
    main_window = Tk()
    label = Label(main_window, text="//Heart Disease Prediction//", font= 'Helvetica 16 bold italic')
    label_2 = Label(main_window, text="•Patient ID\n\n•Age\n\n•Sex\n\n•Chest Pain type\n\n•Resting blood pressure (in mm Hg)\n\n•Cholestoral in mg/dl\n\n•Fasting blood sugar > 120 mg/dl (1 = Yes; 0 = No)\n\n•Resting electrocardiographic results (values 0,1,2)\n\n•Maximum heart rate achieved\n\n•Exercise induced angina (1 = yes; 0 = no)\n\n•ST depression induced by exercise relative to rest\n\n•The slope of the peak exercise ST segment\n\n•Number of major vessels (0-3) colored by flourosopy\n\n•Thal: 0 = normal; 1 = fixed defect; 2 = reversable defect", font='Helvetica 10', justify=LEFT)
    
    values1 = Entry()
    values2 = Entry()
    values3 = Entry()
    values4 = Entry()
    values5 = Entry()
    values6 = Entry()
    values7 = Entry()
    values8 = Entry()
    values9 = Entry()
    values10 = Entry()
    values11 = Entry()
    values12 = Entry()
    values13 = Entry()
    values14 = Entry()
    
    label.place(y=50, x=50)
    label_2.place(y=100, x=50)
    values1.place(y=100, x=450)
    values2.place(y=135, x=450)
    values3.place(y=165, x=450)
    values4.place(y=195, x=450)
    values5.place(y=230, x=450)
    values6.place(y=260, x=450)
    values7.place(y=290, x=450)
    values8.place(y=325, x=450)
    values9.place(y=355, x=450)
    values10.place(y=390, x=450)
    values11.place(y=420, x=450)
    values12.place(y=450, x=450)
    values13.place(y=485, x=450)
    values14.place(y=515, x=450)

    button = Button(main_window, text="Submit", command=lambda: submit_values(main_window,values1,values2, values3, values4, values5, values6, values7, values8, values9, values10, values11, values12, values13, values14))
    button.place(x=50, y=600)
    main_window.geometry('600x700')
    main_window.title("Heart Disease Prediction")
    main_window.mainloop()

In [132]:
def evaluation(Values,data1,data2):
    connection = psycopg2.connect(
    user = "****",
    password = "****",
    host = "localhost",
    port = "5432",
    database = "postgres"
    )

   
    model = load_model('ANN_model_2.h5')
    loaded_model = model.load_weights('ANN_model_2.weights.h5')
    """model.compile(
        loss = keras.losses.BinaryCrossentropy(from_logits=False),
        optimizer = keras.optimizers.Adam(learning_rate=0.0005),
        metrics = ['accuracy'])"""

    scaler = joblib.load('scale.pkl')
    column_names2 = ["age","sex","cp","trestbps","chol","fbs","restecg","thalach","exang","oldpeak","slope",
                 "ca","thal"]
    Values = np.array(Values)
    Values = Values.reshape(1, -1)
    Values = pd.DataFrame(Values,columns=column_names2)
    Values = scaler.transform(Values)
    val_tensor = tf.convert_to_tensor(Values)
    target_arr = model.predict(val_tensor)
    target = target_arr[0][0]
    threshold = 0.5
    if target > threshold:
        target = 1
    else:
        target = 0
    data2.insert(11,target)
    cursor = connection.cursor()
    sql2 = "INSERT INTO hasta_bilgi(hasta_id, age, sex) VALUES(%s, %s, %s)"
    sql3 = "INSERT INTO kalp(cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,hasta_id) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    cursor.execute(sql2,data1)
    cursor.execute(sql3,data2)
    connection.commit()
    connection.close()
   
    

In [133]:
createUI()